In [2]:
import pandas as pd

In [92]:
df = pd.read_csv("./training_set_VU_DM.csv")
df.dropna(axis=1)

,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_brand_bool,prop_location_score1,prop_log_historical_price,...,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,click_bool,booking_bool
0,1,2013-04-04 08:32:15,12,187,219,893,3,1,2.83,4.95,...,23246,1,0,4,0,1,1,1,0,0
1,1,2013-04-04 08:32:15,12,187,219,10404,4,1,2.20,5.03,...,23246,1,0,4,0,1,1,1,0,0
2,1,2013-04-04 08:32:15,12,187,219,21315,3,1,2.20,4.92,...,23246,1,0,4,0,1,1,1,0,0
3,1,2013-04-04 08:32:15,12,187,219,27348,2,1,2.83,4.39,...,23246,1,0,4,0,1,1,1,0,0
4,1,2013-04-04 08:32:15,12,187,219,29604,4,1,2.64,4.93,...,23246,1,0,4,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958342,332785,2013-06-30 19:55:18,5,219,219,77700,3,1,1.61,0.00,...,16974,1,21,3,0,1,0,0,0,0
4958343,332785,2013-06-30 19:55:18,5,219,219,88083,3,1,1.95,0.00,...,16974,1,21,3,0,1,0,0,0,0
4958344,332785,2013-06-30 19:55:18,5,219,219,94508,3,1,1.10,0.00,...,16974,1,21,3,0,1,0,0,0,0
4958345,332785,2013-06-30 19:55:18,5,219,219,128360,3,1,1.95,0.00,...,16974,1,21,3,0,1,0,0,1,1


In [93]:
df = df.dropna(axis=1)
df.columns

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'prop_country_id', 'prop_id', 'prop_starrating', 'prop_brand_bool',
       'prop_location_score1', 'prop_log_historical_price', 'position',
       'price_usd', 'promotion_flag', 'srch_destination_id',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool',
       'random_bool', 'click_bool', 'booking_bool'],
      dtype='object')

In [ ]:
df = pd.read_csv("./training_set_VU_DM.csv")
    df = df.dropna(axis=1)
    merged = pd.merge(
        df[['srch_id','click_bool', 'booking_bool']].groupby('srch_id').sum().reset_index().drop("booking_bool", axis=1).rename(columns={'click_bool':"summed_click_bool"}), 
        df[['srch_id','click_bool', 'booking_bool']], 
        how='left', 
        on= 'srch_id'
    )
    df['rating'] = merged.apply(lambda row: calc_CTR(row['click_bool'], row['booking_bool'], row['summed_click_bool']) ,axis=1)
    df.drop(['date_time', 'srch_id', 'site_id', 'prop_id'])

In [20]:
training_set = df[['srch_id','click_bool', 'booking_bool']]

The formula below is: https://www.desmos.com/calculator/swv4ki1jw3  
$y_{d}=\beta-\frac{1}{bw_{1}+1}$  
$y_{1}=\left\{x<b:\beta-\frac{1}{xw_{1}+1},\frac{1-y_{d}}{m-b}\left(x-b\right)+y_{d}\right\}$  
$y=\left\{\beta=1:\ y_{1},0\right\}$  

In [44]:
def get_CTR(clicks, booking, maximum=25, break_point=5, w_1=2):
    break_score = booking - 1/((break_point*w_1)+1)
    return 0 if booking == 0 else booking - 1/((clicks*w_1)+1) if clicks <= break_point else min(((1-break_score)/(maximum-break_point))*(clicks-break_point)+break_score,1)

Ignore the one above: This is a better scoring metric: https://www.desmos.com/calculator/klyz3aeema

In [83]:
filtered = training_set.groupby('srch_id').sum().reset_index().drop("booking_bool", axis=1).rename(columns={'click_bool':"summed_click_bool"})
merged = pd.merge(filtered, training_set, how='left', on= 'srch_id')

In [84]:
def calc_CTR(click, book, TC, w_1=0.001, w_2=1):
    return min(0 if click == 0 and book == 0 else w_2/TC if click > 0 and book == 1 else w_1*1/TC ,1)

In [86]:
merged['likely_hood'] = merged.apply(lambda row: calc_CTR(row['click_bool'], row['booking_bool'], row['summed_click_bool']) ,axis=1)

In [89]:
res.value_counts()

0.000000    4736468
1.000000     129879
0.000100      56885
0.000050      11666
0.500000       5878
0.000033       5426
0.000025       3048
0.000020       1866
0.000017       1497
0.333333       1417
0.000014        925
0.000013        651
0.250000        552
0.000011        392
0.000009        281
0.200000        264
0.000010        188
0.166667        177
0.000008        124
0.000008        114
0.000006        108
0.000007         94
0.000007         86
0.142857         83
0.125000         53
0.000005         38
0.000006         33
0.111111         31
0.000004         25
0.000004         23
0.000005         19
0.090909         16
0.100000         12
0.083333          6
0.076923          6
0.071429          4
0.066667          4
0.062500          4
0.050000          2
0.058824          1
0.041667          1
dtype: int64

In [45]:
res = training_set.groupby('srch_id').sum().reset_index().apply(lambda row: calc_CTR(row['click_bool'], row['booking_bool']), axis=1)
res

0         0.473684
1         0.000000
2         0.473684
3         0.473684
4         0.000000
            ...   
199790    0.000000
199791    0.473684
199792    0.642857
199793    0.473684
199794    0.473684
Length: 199795, dtype: float64

In [47]:
res.min()

0.0

In [60]:
summed = training_set.groupby('srch_id').sum().reset_index()
rows = summed.shape[0]

In [65]:
print(summed[(summed['click_bool'] > 0) &(summed['booking_bool'] == 0)].shape[0]/rows, 
summed[(summed['click_bool'] > 0) &(summed['booking_bool'] == 1)].shape[0]/rows, 
summed[(summed['click_bool'] == 0) &(summed['booking_bool'] == 0)].shape[0]/rows)

0.3073400235241122 0.6926599764758878 0.0


In [49]:
df[(df['click_bool'] == 1) &(df['booking_bool'] == 0)].shape

(83489, 54)

In [50]:
df[(df['click_bool'] == 1) &(df['booking_bool'] == 1)].shape

(138390, 54)

In [66]:
df[(df['click_bool'] == 0) &(df['booking_bool'] == 0)].shape

(4736468, 54)

In [103]:
df = pd.read_csv("./training_set_VU_DM.csv")

In [107]:
(df.groupby('srch_id')['prop_id'].value_counts() > 1).any()

False

In [97]:
df = df.dropna(axis=1)
df['rating'] = pd.merge(
    df[['srch_id','click_bool', 'booking_bool']].groupby('srch_id').sum().reset_index().drop("booking_bool", axis=1).rename(columns={'click_bool':"summed_click_bool"}), 
    df[['srch_id','click_bool', 'booking_bool']], 
    how='left', 
    on= 'srch_id'
).apply(lambda row: calc_CTR(row['click_bool'], row['booking_bool'], row['summed_click_bool']) ,axis=1)
df = df.drop(['date_time', 'srch_id', 'site_id', 'prop_id'],axis=1)

KeyError: "['date_time', 'srch_id', 'site_id', 'prop_id'] not found in axis"

,visitor_location_country_id,prop_country_id,prop_starrating,prop_brand_bool,prop_location_score1,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,click_bool,booking_bool,rating
0,0.808696,0.951965,0.6,1.0,0.405444,0.797101,0.666667,0.000005,0.0,0.818047,0.0,0.000000,0.375,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.808696,0.951965,0.8,1.0,0.315186,0.809984,0.641026,0.000009,0.0,0.818047,0.0,0.000000,0.375,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,0.808696,0.951965,0.6,1.0,0.315186,0.792271,0.512821,0.000009,0.0,0.818047,0.0,0.000000,0.375,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.808696,0.951965,0.4,1.0,0.405444,0.706924,0.846154,0.000031,0.0,0.818047,0.0,0.000000,0.375,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,0.808696,0.951965,0.8,1.0,0.378223,0.793881,0.076923,0.000007,0.0,0.818047,0.0,0.000000,0.375,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958342,0.947826,0.951965,0.6,1.0,0.230659,0.000000,0.025641,0.000006,0.0,0.597311,0.0,0.042683,0.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4958343,0.947826,0.951965,0.6,1.0,0.279370,0.000000,0.051282,0.000005,0.0,0.597311,0.0,0.042683,0.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4958344,0.947826,0.951965,0.6,1.0,0.157593,0.000000,0.076923,0.000005,0.0,0.597311,0.0,0.042683,0.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4958345,0.947826,0.951965,0.6,1.0,0.279370,0.000000,0.000000,0.000007,0.0,0.597311,0.0,0.042683,0.250,0.0,0.0,0.0,0.0,1.0,1.0,1.0
